# Scraper Outline
* Open Main Page
* Click on name
* Pull Ethnicity
* Pull Rank History
* Click on button2
* Count Rows
* Pull Awards
* Click Button 3
* Count Rows
* Pull Training

In [372]:
# Import Dependencies

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import urllib.request, urllib.error
import csv
import time
import regex as re
from string import ascii_uppercase

In [373]:

# Open initial site

url = "https://oip.nypdonline.org/view/1004/@SearchName=SEARCH_FILTER_VALUE&@LastNameFirstLetter=A//%7B%22hideMobileMenu%22:true%7D/true/true"
driver = webdriver.Chrome()
driver.get(url) # Go to NYPD page


In [374]:
# Open the Frame
wrapper = driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/report-wrapper/report/div/div/div[2]/div/div/div[2]/table/tbody/tr[8]")
wrapper.click()
time.sleep(5)
new_frame = driver.find_element_by_tag_name("iframe")

In [375]:
# Switch to new driver
driver.switch_to.frame(new_frame)

In [210]:
# Get Ethnicity
ethnicity = driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[1]/report-wrapper/report/div/div/div[2]/div/div[2]/div[2]/div/div[5]/div[2]")
print(ethnicity.text)

HISPANIC


In [263]:
## RANK & SHIELD HISTORY
# Click Button
driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/report-header/div/span/span/button[1]").click()
time.sleep(3)

# Get Rank & Shield History
officer_name = []
date = []
title = []
shield_no = []
rows = int(len(driver.find_elements_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div/div[2]/div/div/div[2]/table/tbody/tr")))

for row in range(1,rows + 1):
    officer_name.append(driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[1]/report-wrapper/report/div/div/div[2]/div/div[1]").text)
    date.append(driver.find_element_by_xpath(f"/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div/div[2]/div/div/div[2]/table/tbody/tr[{row}]/td[1]").text)
    title.append(driver.find_element_by_xpath(f"/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div/div[2]/div/div/div[2]/table/tbody/tr[{row}]/td[2]").text)
    shield_no.append(driver.find_element_by_xpath(f"/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div/div[2]/div/div/div[2]/table/tbody/tr[{row}]/td[3]").text)

# print(officer_name)
# print(date)
# print(title)
# print(shield_no)

rank_dict =  {"Officer Name": officer_name, "Title": title, "Effective Date": date, "Shield Number": shield_no}

rank_df = pd.DataFrame(rank_dict)
rank_df.head()


,Officer Name,Title,Effective Date,Shield Number
0,"ABBONDANDELO, BRIAN R",POLICE OFFICER,7/9/2014,15447


In [267]:
# Department Recgns & Awards
# Click the button

driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/report-header/div/span/span/button[2]").click()
time.sleep(2)

officer_name = []
date = []
award = []
rows = int(len(driver.find_elements_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div/div[2]/div/div/div[2]/table/tbody/tr")))

try:
    for row in range(1,rows + 1):
        officer_name.append(driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[1]/report-wrapper/report/div/div/div[2]/div/div[1]").text)
        date.append(driver.find_element_by_xpath(f"/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div/div[2]/div/div/div[2]/table/tbody/tr[{row}]/td[1]").text)
        award.append(driver.find_element_by_xpath(f"/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div/div[2]/div/div/div[2]/table/tbody/tr[{row}]/td[2]").text)
        
except:
    print("No Recognitions")

print(officer_name)
print(date)
print(award)

award_dict =  {"Officer Name": officer_name, "Award Date": date, "Award": award}

award_df = pd.DataFrame(award_dict)
award_df.head()



['ABBONDANDELO, BRIAN R', 'ABBONDANDELO, BRIAN R', 'ABBONDANDELO, BRIAN R']
['4/2/2019', '1/28/2019', '6/5/2017']
['EXCELLENT POLICE DUTY', 'EXCELLENT POLICE DUTY', 'EXCELLENT POLICE DUTY']


,Officer Name,Award Date,Award
0,"ABBONDANDELO, BRIAN R",4/2/2019,EXCELLENT POLICE DUTY
1,"ABBONDANDELO, BRIAN R",1/28/2019,EXCELLENT POLICE DUTY
2,"ABBONDANDELO, BRIAN R",6/5/2017,EXCELLENT POLICE DUTY


In [236]:
# Get Arrests Processed
driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/report-header/div/span/span/button[5]").click()
time.sleep(2)
arrests = driver.find_elements_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div/div[2]/div/div/div[2]/table/tbody/tr")

print(len(arrests))


0


In [301]:
# Click Button
driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/report-header/div/span/span/button[5]").click()
time.sleep(4)

# Get Length of rows of table
rows = len(driver.find_elements_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div/div[2]/div/div/div[2]/table/tbody/tr"))

type = []
amount = []
officer_name = []

for row in (1, rows):
    officer_name.append(driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[1]/report-wrapper/report/div/div/div[2]/div/div[1]").text)
    type.append(driver.find_element_by_xpath(f"/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div/div[2]/div/div/div[2]/table/tbody/tr[{row}]/td[1]").text)
    amount.append(driver.find_element_by_xpath(f"/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div/div[2]/div/div/div[2]/table/tbody/tr[{row}]/td[2]").text)

print(officer_name)
print(type)
print(amount)

arrests_dict =  {"Officer Name": officer_name, "Classification": type, "Total Arrests": amount}

arrests_df = pd.DataFrame(arrests_dict)
arrests_df.head()



['ABBONDANDELO, BRIAN R', 'ABBONDANDELO, BRIAN R']
['MISDEMEANOR', 'VIOLATION']
['51', '1']


,Officer Name,Classification,Total Arrests
0,"ABBONDANDELO, BRIAN R",MISDEMEANOR,51
1,"ABBONDANDELO, BRIAN R",VIOLATION,1


In [ ]:
# Get Diciplinary History
driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/report-header/div/span/span/button[4]").click()
time.sleep(2)
disc = driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div/div[2]/div/div/div[2]/table")

print(disc.text)

In [ ]:
# Get Documents
driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/report-header/div/span/span/button[6]").click()
time.sleep(2)
docs = driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div/div[2]/div/div/div[2]/table")

print(docs.text)

In [376]:
# Get Training Summary
# Click the button
driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/report-header/div/span/span/button[3]").click()
time.sleep(2)

date = []
training = []
officer_name = []

# Get Number of clicks
pager_info = driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div/div[2]/div/div/div[3]/span").text
pager_info = re.findall(r'\d+(?:,\d+)?',f"{pager_info}") # Regex to isolate the numbers
num_rows = int(pager_info[-1].replace(',','')) # Take the last value, remove the comma & turn the string to an integer
num_clicks = (-(-num_rows//100)) - 1 # Knowing there are 100 entries/page, get number of clicks and round up and subtract 1

for click in range(1,num_clicks + 2):
    # Get Num Rows
    pager_info = driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div/div[2]/div/div/div[3]/span").text
    # pager_info = driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div[2]/div[2]/div/div/div/div/div/div/div[3]/span").text
    pager_info = re.findall(r'\d+(?:,\d+)?',f"{pager_info}")
    beg = int(pager_info[0])
    end = int(pager_info[1])
    items = end - beg + 1

    for item in range(1,items + 1):
        # Get Training
        officer_name.append(driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[1]/report-wrapper/report/div/div/div[2]/div/div[1]").text)
        date.append(driver.find_element_by_xpath(f"/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div/div[2]/div/div/div[2]/table/tbody/tr[{item}]/td[1]").text)
        training.append(driver.find_element_by_xpath(f"/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div/div[2]/div/div/div[2]/table/tbody/tr[{item}]/td[2]").text)

    if click < num_clicks + 1:
        # Go to next page
        driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div/div[2]/div/div/div[3]/a[3]").click()
        time.sleep(2)
    else:
        end  
        
#print(officer_name)
#print(date)
#print(training)
training_results = [officer_name, date, training]

training_dict = {"Officer Name": officer_name, "Date": date, "Training": training}
training_df = pd.DataFrame(training_dict)
training_df.head()
len(training_df)



241

In [377]:
training_df.head(end)

,Officer Name,Date,Training
0,"ABADIR, MINA E",4/6/2022,CLT-PROCEDURAL OPERATIONS
1,"ABADIR, MINA E",4/6/2022,"INTEGRATING COMMUNICATION, ASSESSMENT AND TACT..."
2,"ABADIR, MINA E",3/29/2022,CLT-BWC POLICY UPDATES
3,"ABADIR, MINA E",3/18/2022,SERGEANTS DEVELOPMENT COURSE
4,"ABADIR, MINA E",3/15/2022,"INTEGRATING COMMUNICATION, ASSESSMENT AND TACT..."
...,...,...,...
236,"ABADIR, MINA E",4/12/2012,THREE WHEEL SCOOTER CERTIFIED
237,"ABADIR, MINA E",1/9/2012,FOUR WHEEL DRIVE TRAINING
238,"ABADIR, MINA E",1/9/2012,THREE WHEEL SCOOTER CERTIFIED
239,"ABADIR, MINA E",1/9/2012,12 PASSENGER VAN / UTILITY VEHICLE QUALIFICATION


In [370]:
# switch back to the previous content (YOU MUST DO THIS BEFORE CLOSING THE FRAME)
driver.switch_to.default_content()

In [371]:
# Close the frame
close_icon = driver.find_elements_by_class_name("icon-close")
print(close_icon)
close_icon[1].click()

[<selenium.webdriver.remote.webelement.WebElement (session="5089deeb6209471ed1e9a3242b284bea", element="a61ba7ad-37b8-4057-8489-772af9a0d14e")>]


IndexError: list index out of range

In [ ]:
# Figure out how many times to loop
# Create this into a function to call this whenever you go to a new letter of the the last name

pager_info = driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/report-wrapper/report/div/div/div[2]/div/div/div[3]/span").text
pager_info = re.findall(r'\d+(?:,\d+)?',f"{pager_info}")
beg = int(pager_info[0])
end = int(pager_info[1])
num_rows = end - beg + 1
print(num_rows)

In [272]:
# Click Next Page (Training Page)
driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div/div[2]/div/div/div[3]/a[3]").click()


In [273]:
pager_info = driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div/div[2]/div/div/div[3]/span").text
pager_info = re.findall(r'\d+(?:,\d+)?',f"{pager_info}") # Regex to isolate the numbers
num_rows = int(pager_info[-1].replace(',','')) # Take the last value, remove the comma & turn the string to an integer
num_clicks = (-(-num_rows//100)) - 1 # Knowing there are 100 entries/page, get number of clicks and round up and subtract 1
print(num_clicks)

2


In [338]:
# Click on Diciplinary History
driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/report-header/div/span/span/button[4]").click()
officer_name = []
date = []
charges = []
allegations = []

try:
    officer_name.append(driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[1]/report-wrapper/report/div/div/div[2]/div/div[1]").text)
    date.append(driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div[2]/div[2]/div/div/div/div/div/div/div[2]/table/tbody/tr/td[1]").text)
    charges.append(driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div[2]/div[2]/div/div/div/div/div/div/div[2]/table/tbody/tr/td[2]").text)
    allegations.append(driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div[2]/div[2]/div/div/div/div/div/div/div[2]/table/tbody/tr/td[3]").text)

except:
    print("No Diciplinary History")

print([officer_name,date,charges,allegations])

[['PABON, ELIEZER'], ['1/31/2015'], ['1'], ['0']]


In [336]:
def get_diciplinary_history(driver):
        # Click on Diciplinary History
        driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/report-header/div/span/span/button[4]").click()
        officer_name = []
        date = []
        charges = []
        allegations = []

        try:
            officer_name.append(driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[1]/report-wrapper/report/div/div/div[2]/div/div[1]").text)
            date.append(driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div[2]/div[2]/div/div/div/div/div/div/div[2]/table/tbody/tr/td[1]").text)
            charges.append(driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div[2]/div[2]/div/div/div/div/div/div/div[2]/table/tbody/tr/td[2]").text)
            allegations.append(driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div[2]/div[2]/div/div/div/div/div/div/div[2]/table/tbody/tr/td[3]").text)

        except:
            print("No Diciplinary History")

        diciplinary_results = [officer_name,date,charges,allegations]

        return diciplinary_results

In [345]:
diciplinary_results = get_diciplinary_history(driver)
print(diciplinary_results)

No Diciplinary History
[['PABON, NELSON'], [], [], []]


In [346]:
def write_to_csv(results):
    # Get Size
    
    # Create Dictionaries
    ethnicity_dict = {"Officer Name": ethnicity_results[0][:], "Ethnicity": ethnicity_results[1][:]}
    print(ethnicity_results)
    
    # Write to CSV
        
    return 0

In [353]:
def get_ethnicity(driver):
    ethnicity = driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[1]/report-wrapper/report/div/div/div[2]/div/div[2]/div[2]/div/div[5]/div[2]").text
    officer_name = driver.find_element_by_xpath("/html/body/section/div/div/section/div/div/div[2]/div/div/div[1]/report-wrapper/report/div/div/div[2]/div/div[1]").text

    ethnicity_results = [officer_name, ethnicity]
    return ethnicity_results

In [354]:
ethnicity_results = get_ethnicity(driver)
print(ethnicity_results)

['PABON, NELSON', 'HISPANIC']


In [363]:
len(ethnicity_results[0])

13

In [ ]:
/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div[2]/div[2]/div/div/div/div/div/div/div[3]/span
/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div[2]/div[2]/div/div/div/div/div/div/div[3]/span
/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div/div[2]/div/div/div[3]/span
/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div[2]/div[2]/div/div/div/div/div/div/div[3]/span
/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div/div[2]/div/div/div[3]/span
/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div[2]/div[2]/div/div/div/div/div/div/div[3]/span
/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div[2]/div[2]/div/div/div/div/div/div/div[3]/span
/html/body/section/div/div/section/div/div/div[2]/div/div/div[2]/report-wrapper/report/div/div[2]/div[2]/div/div/div/div/div/div/div[3]/span